In [1]:
import os
import cv2
import math
import random
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
from torch.autograd import Variable
from torchvision import datasets, transforms, models
from sklearn.metrics import classification_report
from torch import optim
import time
from utils import Conv2d

In [2]:
# parameters setting
input_shape = [3,96,96]
num_classes = 7
num_layers = 5
cut_size = 96
init_lr = 0.0003
batch_size = 256
use_fer=True
device= torch.device("cuda")

In [3]:
# load data
data_dir = ''

if use_fer:
    full_data = torch.load(data_dir + 'pickles/fer2013.pt')
    full_img, full_emo = full_data['images'][:,None,:,:], full_data['label']

    train_img,train_y = full_img[:28709],full_emo[:28709]
    test_img,test_y = full_img[28709:28709+3589],full_emo[28709:28709+3589]

    train_img = torch.cat((train_img,train_img,train_img),1)
    test_img = torch.cat((test_img,test_img,test_img),1)

else:
    train_data = torch.load(data_dir + 'pickles/AffectNet_train.pt')
    train_img, train_y = train_data['images'], train_data['emotion']
    
    test_data = torch.load(data_dir + 'pickles/AffectNet_test.pt')
    test_img, test_y = test_data['images'], test_data['emotion']
    

In [4]:
from torch.utils.data import Dataset, DataLoader
#======= data loaders =======#    

train_transform = transforms.Compose([
    transforms.ToPILImage(),
#     transforms.Grayscale(num_output_channels=3),
    transforms.Resize([int(cut_size*1.1),int(cut_size*1.1)]),
    transforms.RandomCrop([cut_size,cut_size]),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
#     transforms.Grayscale(num_output_channels=3),
    transforms.Resize([int(cut_size*1.1),int(cut_size*1.1)]),
    transforms.CenterCrop([cut_size,cut_size]),
    transforms.ToTensor()
])

def default_loader(images):
    img_tensor = train_transform(images)
    return img_tensor

def default_test_loader(images):
    img_tensor = test_transform(images)
    return img_tensor

class Dataset(Dataset):
    def __init__(self,imgs,ys,loader=default_loader):
        self.images = imgs 
        self.label = ys
        self.loader = loader

    def __getitem__(self, index):

        fn = self.images[index]
        img = self.loader(fn)
        label = self.label[index]
        return img,label

    
    def __len__(self):
        return len(self.images)

In [7]:
class AutoEncoder(nn.Module):
    def __init__(self, num_layers, batch_size, target_shape, init_lr):
        super(AutoEncoder, self).__init__()
        self.name = 'AutoEncoder_{}'.format(num_layers)
        print(self.name)
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.im_shape = target_shape
        self.dec_im = None
        self.imgs_train = None
        
        self.encoder = nn.Sequential(
            # conv_0
            Conv2d(in_channels=target_shape[0], out_channels=32, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            # conv_1
            Conv2d(in_channels=32, out_channels=64, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            # conv_2
            Conv2d(in_channels=64, out_channels=128, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            # conv_3
            Conv2d(in_channels=128, out_channels=256, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            # conv_4
            Conv2d(in_channels=256, out_channels=512, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.BatchNorm2d(512)
        )
        
        if num_layers==5:
            self.encoder_ = nn.Sequential(
                nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=2, stride=2),
                nn.ReLU,
                nn.BatchNorm2d(1024)
            )
            
            self.decoder_ = nn.Sequential(
                F.interpolate(scale_factor=2, mode='bilinear', align_corners=True),
                nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=3, stride=2),
                nn.ReLU,
                nn.BatchNorm2d(512)
            )
        
        self.decoder = nn.Sequential(
            # D2            
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            Conv2d(in_channels=512, out_channels=256, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            # D3
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            Conv2d(in_channels=256, out_channels=128, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            # D4
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            Conv2d(in_channels=128, out_channels=64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            # D5
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            Conv2d(in_channels=64, out_channels=32, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),

            Conv2d(in_channels=32, out_channels=target_shape[0],kernel_size=3, stride=1),
            nn.Tanh(),
        )
            
            
        self.optimizer = torch.optim.Adam(params=[p for p in self.parameters() if p.requires_grad], lr=init_lr, eps=1e-5)
        self.loss_fn = self.ae_loss()

    
    def ae_loss(self, tower=0):
        ae_loss = nn.MSELoss(self.imgs_train, self.dec_im)
        return ae_loss
    
    def model(num_layers):
        if num_layers==5:
            self.model = nn.Sequential(self.encoder, self.encoder_, self.decoder_, self.decoder)
        else:
            self.model = nn.Sequential(self.encoder, self.decoder)
    
    def train_model(self, data_loader, num_epochs, lr):
        self.train()
        start=time.time()
        
        total_loss = 0
        
        for epoch in range(num_epochs):
            running_loss = 0.
            total = 0
            total_dec_im = []
            total_raw_im = []
            num_batches = 0
            for count, (images, label) in enumerate(data_loader):
                self.optimizer.zero_grad()
                
                scores = 0.
                total_raw_im.append(images)
                images = images.to(device)
#                 label = label.to(device)
                
                enc_im = self.encoder(images)
                
                self.dec_im = self.decoder(enc_im)  
                
        
                loss = self.loss_fn(self.dec_im, images)
                loss.backward()
                
                self.optimizer.step()
                
                total_dec_im.append(self.dec_im.detach().cpu())
                running_loss += loss.detach().cpu().item()
                total += images.size(0)
                num_batches += 1
                
            total_loss = running_loss/num_batches
            elapsed = time.time()-start
            
            print('\t', 'epoch=',epoch, '\t time = {:.0f}m {:.0f}s'.format(elapsed // 60, elapsed % 60), 
                  '\t lr=', lr, '\t loss = ', total_loss )
            
            total_dec_im = torch.cat(total_dec_im,0)
            total_raw_im = torch.cat(total_raw_im,0)
        return total_dec_im, total_raw_im, total_loss

In [8]:
class transfer(nn.Module):

    def __init__(self,model=None,lr=0.0001,num_classes=7):
        super(transfer, self).__init__()
        self.model = model
        
        for p in self.model.encoder.parameters():
                p.requires_grad = False
        
        for p in self.model.decoder.parameters():
                p.requires_grad = False
        
        self.classifier1 = nn.Sequential(
            nn.Conv2d(512,1024,3,1),
            nn.ReLU(),
            nn.BatchNorm2d(1024),
            nn.Conv2d(1024,1024,3,1),
            nn.ReLU(),
            nn.BatchNorm2d(1024),
        )
        self.classifier2 = nn.Sequential(
            nn.Linear(4096,1024),
            nn.ReLU(),
#             nn.Linear(1024,256),
#             nn.ReLU(),
            nn.Linear(1024,7)
        )
        
        self.loss_fn = nn.CrossEntropyLoss() #FocalLoss(4.0)
        self.optimizer=torch.optim.Adam([p for p in self.parameters() if p.requires_grad] , lr=lr)
        

    def train_model(self, data_loader, num_epochs, lr):
        self.train()
        start=time.time()

        for epoch in range(num_epochs):
            running_loss=0
            running_acc = 0.
            total = 0
            num_batches = 0
            for count, (images, label) in enumerate(data_loader):
                self.optimizer.zero_grad()

                scores = 0.
                
                images=images.to(device)

                enc_im = self.model.encoder(images)
                x = self.classifier1(enc_im).view(images.size(0),-1)
                scores = self.classifier2(x)

                label=label.to(device)
                loss =  self.loss_fn(scores, label) 
                loss.backward()

                self.optimizer.step()

                running_loss += loss.detach().item()

                acc = (scores.argmax(-1) == label).float().sum()
                running_acc += acc.item()

                total+=images.size(0)
                num_batches += 1
                
            total_loss = running_loss/num_batches
            total_acc = running_acc/total
            elapsed = time.time()-start
            print('\t', 'epoch=',epoch, '\t time = {:.0f}m {:.0f}s'.format(elapsed // 60, elapsed % 60),'\t lr=', lr,'\t loss = ', total_loss , '\t Acc = {:.6f}'.format(total_acc*100),'%')

    def test_model(self, data_loader):
        self.eval()
        start=time.time()
        
        running_loss = 0.
        running_acc = 0
        total = 0
        num_batches = 0
        
        preds = []
        truth = []
        total_enc_im = []
        total_dec_im = []
        
        for count, (images, label) in enumerate(data_loader):
            self.optimizer.zero_grad()
            scores = 0.
            images = images.to(device)

            enc_im = self.model.encoder(images)
            dec_im = self.model.decoder(enc_im)  
            x = self.classifier1(enc_im).view(images.size(0),-1)
            scores = self.classifier2(x).detach().cpu()

            acc = (scores.argmax(-1) == label).float().sum()
            running_acc += acc.item()
            
            total += images.size(0)
            num_batches += 1
            
            total_enc_im.append(enc_im.detach().cpu())
            total_dec_im.append(dec_im.detach().cpu())

            preds.extend(scores.argmax(-1).numpy())
            truth.extend(label.numpy())
            
        total_acc = running_acc/total
        elapsed = time.time()-start

        print('testing','\t time = {:.0f}m {:.0f}s'.format(elapsed // 60, elapsed % 60), '\n Acc = {:.6f}'.format(total_acc*100),'%')
        test_fscore = classification_report(truth, preds)
        print(test_fscore)

        total_dec_im = torch.cat(total_dec_im,0)
        total_enc_im = torch.cat(total_enc_im,0)
        return total_enc_im, total_dec_im, preds

In [ ]:
batch_size = 256
init_lr=0.0003

train_input  = Dataset(train_img,train_y)
train_loader = DataLoader(train_input, batch_size=batch_size,shuffle=False)

test_input  = Dataset(test_img,test_y,default_test_loader)
test_loader = DataLoader(test_input, batch_size=batch_size,shuffle=False)

target_shape=[train_input.images.size(1), 96, 96]

model = AutoEncoder(num_layers=4, batch_size=batch_size, target_shape=target_shape, init_lr=init_lr)
model.load_state_dict(torch.load(data_dir + 'baselines/AE_AffectNet_inputsize96_eph400.pth'))

net4 = transfer(model=model)
net4.to(device)
for i in range(20):
    print('Iteration', i)
    net4.train_model(data_loader=train_loader, num_epochs=5,lr=init_lr)
    test_enc_im4, test_dec_im4, preds4=net4.test_model(data_loader=test_loader)